# Assignment Data Structure and Solution
#### We can use this Jupyter Notebook to analysis the data structure of different file and variables to help us to solve these three model construction and analysis.

# First Class(CommandLine)
<big>**Calss Function**</big>:
   1. Judge the command line to accord with the stationary Formate.
   2. Print Help Inforamtion if the command formate is wrong.
   3. Default:<br>
      1)'index_nostoplist_nostemming.txt' / queries_nostoplist_nostemming.txt'. <br>
      2) For the First dataset(index?), it's the key vocabularly in the text. <br>
      3) For the second dataset(queries?), it's the summary of all the search queris. <br>
       
**Example Command Formate**:
```
> python ir_engine.py -o result.txt
```

In [9]:
#==============================================================================
# Importing
import sys, getopt, re
import time
#==============================================================================
# Command line processing
class CommandLine:
    def __init__(self):
        opts, args = getopt.getopt(sys.argv[1:], 'hspw:o:')
        opts = dict(opts)
        self.exit = True

        if '-h' in opts:
            self.printHelp()
            return

        if len(args) > 0:
            print("*** ERROR: no arg files - only options! ***", file=sys.stderr)
            self.printHelp()
            return

        if '-w' in opts:
            if opts['-w'] in ('binary', 'tf', 'tfidf'):
                self.termWeighting = opts['-w']
            else:
                warning = (
                    "*** ERROR: term weighting label (opt: -w LABEL)! ***\n"
                    "    -- value (%s) not recognised!\n"
                    "    -- must be one of: binary / tf / tfidf"
                    )  % (opts['-w'])
                print(warning, file=sys.stderr)
                self.printHelp()
                return
        else:
            self.termWeighting = 'binary'

        if '-o' in opts:
            self.outfile = opts['-o']
        else:
            print("*** ERROR: must specify output file (opt: -o FILE) ***",
                  file=sys.stderr)
            self.printHelp()
            return

        if '-s' in opts and '-p' in opts:
            self.indexFile   = 'index_withstoplist_withstemming.txt'
            self.queriesFile = 'queries_withstoplist_withstemming.txt'
        elif '-s' in opts:
            self.indexFile   = 'index_withstoplist_nostemming.txt'
            self.queriesFile = 'queries_withstoplist_nostemming.txt'
        elif '-p' in opts:
            self.indexFile   = 'index_nostoplist_withstemming.txt'
            self.queriesFile = 'queries_nostoplist_withstemming.txt'
        else:
            self.indexFile   = 'index_nostoplist_nostemming.txt'
            self.queriesFile = 'queries_nostoplist_nostemming.txt'
            
        self.exit = False

    def printHelp(self):
        progname = sys.argv[0]
        progname = progname.split('/')[-1] # strip off extended path
        help = __doc__.replace('<PROGNAME>', progname, 1)
        print(help, file=sys.stderr)


# Second Class(IndexLoader)
<big>**Calss Function**</big>:
   1. Use the **Regular Expression** to extract the information from original file and index in a new dictionary.
   2. The New Generator dictionary has 2 dimensions.

<big>**Data Structure(Python Dictionary)**</big>:<br>
   1. {(term name)->{(term name)-> Count}}

In [10]:
#==============================================================================
# Load (precomputed) Index File for (preprocessed) Document Collection
class IndexLoader:
    def __init__(self, indexFile):
        self.index = {}
        docidCountRE = re.compile('(\d+):(\d+)')
        f = open(indexFile, 'r')
        for line in f:
            term = line.split(' ', 1)[0]
            self.index[term] = {}
            for (docid, count) in docidCountRE.findall(line):
                docid = int(docid)
                self.index[term][docid] = int(count)
    def getIndex(self):
        return self.index

# Third Class(Queries)
<big>**Calss Function**</big>:
   1. Use the **Regular Expression** to extract the information from original file and index in a new dictionary.
   2. The New Generator dictionary has 2 or more dimensions.
   3. This class is to extract the queries and get the same dataset formate(python dictionary) as IndexLoader

<big>**Data Structure(Dictionary)**</big>:<br>
   1. {(qid name)->{(qid name)-> count_num}}

In [11]:
class Queries:
    def __init__(self, queriesFile):
        self.qStore = {}
        termCountRE = re.compile('(\w+):(\d+)')
        f = open(queriesFile, 'r')
        for line in f:
            qid = int(line.split(' ', 1)[0])
            self.qStore[qid] = {}
            for (term, count) in termCountRE.findall(line):
                self.qStore[qid][term] = int(count)
    
    def getQuery(self, qid):
        if qid in self.qStore:
            return self.qStore[qid]
        else:
            print("*** ERROR: unknown query identifier (\"%s\") ***" % qid, file=sys.stderr)
            if type(qid) == type(''):
                print('WARNING: query identifiers should be of type: integer', file=sys.stderr)
                print('         -- your query identifier is of type: string', file=sys.stderr)
            print(' -- program exiting', file=sys.stderr)
    
    def qids(self):
        return sorted(self.qStore)

# Forth Class(ResultStore)
<big>**Calss Function**</big>:<br>
<big>**Data Structure(Dictionary)**</big>:<br>

In [12]:
class ResultStore:
    def __init__(self, outfile):
        self.outfile = outfile
        self.results = []

    def store(self, qid, docids):
        if len(docids) > 10:
            docids = docids[:10]
        self.results.append((qid, docids))

    def output(self):
        with open(self.outfile, 'w') as out:
            for (qid, docids) in self.results:
                for docid in docids:
                    print(qid, docid, file=out)

# Target Class(Retrieve)

In [5]:
# TODO: Finish the Assignment Part
import math

class Retrieve:
    # Define all the usable class variables
    index = None
    termWeighting = None
#     IDF_value = dict()  # IDF values for each documents
    doc_len = dict()  # The dictionary to record all the documents' length
    tr_candidate = dict()  # Retrieval all the documents 
    tr_candidate_all = dict() # for every doc_id and count
    tr_candidate_id = list()  # Retrial all the documents ID and store for use binary model candidate
    tr_query_count = list()  # To store the query count/frequency

    # Create new Retrieve object storing index and termWeighting scheme
    def __init__(self, index, termWeighting):
        self.index = index
        self.termWeighting = termWeighting
        # get the length of the each documents and
        self.doc_length()

    # Firstly we should calculate the documents length by number of arisen vocabularly
    def doc_length(self):
        # Extract all the items from the index file and sum the arisen number
        tempor_list = self.index.items()
        for term,combination in tempor_list:
            for doc_id,show_count in combination:
                if doc_id in self.doc_len:
                    self.doc_len[doc_id] = self.doc_len[doc_id] + int(show_count)
                else:
                    self.doc_len[doc_id] = int(show_count)

    # Method performing retrieval for specified query
    def forQuery(self, query):
        # Compare the index and query
        self.getCandidate(index, query) 
        # Firstly we should judge the function the program operate
        if self.termWeighting is 'binary':
            output = self.binary_model(query)
        
        return output
        

    # Extract the query from the file and also generate a new corresponding dictionary
    # We delete all the irreverent values and integrate the dictionary into a list(Count all the count)
    def getCandidate(self, index, query):
        # Get the two list to store the term of query and number of key vocabularly
        queryIterm = list(query.keys())
        qtermCount = list(query.values())
        for each_iterm in set(queryIterm):            
            if each_iterm in index:   # If the target document includes the query key we should put the id into a document
                self.tr_candidate[each_iterm] = index[each_iterm] # get all the document data formate: doc_id:count
                for doc_id,count in index[each_iterm]:
                    if doc_id in self.tr_candidate_all:
                        self.tr_candidate_all[doc_id][each_iterm] = count
                    else:
                        self.tr_candidate_all[doc_id] = 0
                id_list = list(index[each_iterm].keys())
                self.tr_candidate_id.extend(id_list)         

    def binary_model(self, query):
        file_len = self.doc_len
        Results = dict()
        # Calculate all the value from the self.tr_documentid and convert it into the set to improve the efficiency
        # We have built the tr_candidate_id to store all the show index file and number is the occurance of query     
        # We need to calculate the number of occurance of index in set of query
        binary_candidate = set(self.tr_candidate_id)
        score = 0        
        for doc_id in binary_candidate.items():
            for each_term,each_value in (query.iterms()):
                if each_term in self.tr_candidate_all[doc_id]:
                    score = score + self.tr_candidate_all[doc_id][each_term] * each_value
                else:
                    continue
            result = score / math.sqrt(self.doc_len[candidateID])
            Results[doc_id] = result
            score = 0
        # return value is a list value
        topscore = [count for doc_id,value in sorted(Result.iterms(), key = operator.itemgetter(1), reverse = True)]
        return topscore
               

<big>**Below Code is the Formate of Command(Please customize with on the basis of your requirements)**</big><br><br>

**How to generate command line As you wish**<br>

`sys.argv` is the python reserved variables and applied to `ArgumentParser.parse_args(args=None, namespace=None)` to be the one of the parameters. This is a list of strings, each of which is obviously derived from command line arguments, separated by spaces. We can redefine this list by below command.

We should delete the key parameters like `python` and put the remains into the list.

In [6]:
# Example Command Formate:
#  > python ir_engine.py -o result.txt
sys.argv = ['ir_engine.py', '-o', 'result.txt']

In [19]:
config = CommandLine()
index = IndexLoader(config.indexFile).getIndex()
retrieve = Retrieve(index, config.termWeighting)
queries = Queries(config.queriesFile)
allResults = ResultStore(config.outfile)

In [52]:
index

{'a': {1: 1,
  11: 2,
  14: 2,
  16: 1,
  17: 1,
  20: 1,
  21: 1,
  23: 1,
  25: 1,
  26: 1,
  30: 1,
  33: 2,
  35: 1,
  36: 1,
  38: 1,
  39: 4,
  40: 8,
  42: 1,
  43: 1,
  44: 1,
  45: 1,
  46: 3,
  48: 6,
  54: 2,
  57: 1,
  58: 2,
  60: 1,
  65: 1,
  66: 2,
  68: 2,
  69: 1,
  70: 3,
  71: 9,
  73: 1,
  74: 1,
  76: 1,
  78: 3,
  79: 1,
  80: 1,
  82: 2,
  83: 1,
  87: 2,
  89: 1,
  90: 1,
  91: 1,
  92: 4,
  93: 3,
  94: 2,
  95: 1,
  96: 4,
  102: 1,
  103: 1,
  104: 2,
  105: 1,
  106: 4,
  107: 1,
  109: 1,
  111: 1,
  114: 1,
  116: 4,
  118: 1,
  120: 1,
  121: 1,
  122: 2,
  123: 4,
  124: 1,
  134: 1,
  136: 1,
  138: 1,
  142: 1,
  143: 2,
  144: 1,
  146: 3,
  147: 1,
  149: 1,
  153: 1,
  154: 1,
  161: 1,
  162: 1,
  164: 1,
  165: 1,
  166: 1,
  170: 1,
  172: 1,
  173: 1,
  174: 1,
  175: 1,
  176: 1,
  177: 2,
  184: 2,
  185: 5,
  189: 1,
  191: 1,
  192: 2,
  193: 1,
  195: 1,
  198: 1,
  202: 6,
  203: 1,
  205: 2,
  208: 1,
  209: 1,
  213: 1,
  215: 1,
  216:

In [ ]:
for qid in queries.qids():
    query = queries.getQuery(qid)
    results = retrieve.forQuery(query)
    allResults.store(qid, results)

allResults.output()